In [1]:
import sys
from outletsBiasRatings import outletsBiasRatingsAllSides,outletAbbreviationToFullName
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle
import time
from joblib import Parallel, delayed
from transformers import AutoTokenizer, AutoModelWithLMHead, pipeline

In [2]:
outletsBiasRatings = outletsBiasRatingsAllSides
outlets = list(outletsBiasRatings.keys())
startYear = 2000
endYear = 2019
years=[str(year) for year in range(startYear,endYear+1)]
USE_GPU=1
BATCH_SIZE=64
HEADLINE_MAX_LENGTH=32
print(years)
print(len(outlets))
print(outlets)

['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
47
['alternet', 'democracynow', 'db', 'hp', 'theintercept', 'jacobin', 'motherjones', 'thenewyorker', 'thenation', 'slate', 'vox', 'cnn', 'nyt', 'abcnews', 'theatlantic', 'buzzfeed', 'cbs', 'economist', 'guardian', 'nbcnews', 'politico', 'timemagazine', 'wp', 'npr', 'ap', 'bbc', 'bloomberg', 'csm', 'reuters', 'thehill', 'usatoday', 'wsj', 'reason', 'we', 'wt', 'fox', 'americanspectator', 'bre', 'theblaze', 'cbn', 'dailycaller', 'dailymail', 'dailywire', 'thefederalist', 'nationalreview', 'nyp', 'newsmax']


In [3]:
# j-hartmann/emotion-english-distilroberta-base
# https://huggingface.co/j-hartmann/emotion-english-distilroberta-base?text=Oh+wow.+I+didn%27t+know+that.

## Download model for the first time
# tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")
# model = AutoModelWithLMHead.from_pretrained("j-hartmann/emotion-english-distilroberta-base")

## Save model to local folder
# model.save_pretrained("./j-hartmann/emotion-english-distilroberta-base")
# tokenizer.save_pretrained("./j-hartmann/emotion-english-distilroberta-base")

# Loading model
sentimentPipeline = pipeline(task='sentiment-analysis',model="j-hartmann/emotion-english-distilroberta-base",
                             device=0)#gpu
sentimentPipeline.save_pretrained('./j-hartmann/emotion-english-distilroberta-base')

Some weights of RobertaModel were not initialized from the model checkpoint at j-hartmann/emotion-english-distilroberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
predictions=sentimentPipeline("I'm so excited about the news")
print( predictions )
def get_emotion(batch,USE_GPU):
    try:
        predictions=sentimentPipeline(batch)
        predictionsEmotions=[]
        for prediction in predictions:
            predictionsEmotions.append(prediction['label'])
        return predictionsEmotions
    except Exception as e:
        raise
        print(e)  
        return None

[{'label': 'joy', 'score': 0.9540702104568481}]


In [6]:
def annotateEmotionsInOutletHeadlines(outlet,startYear,endYear):
    print(outlet,end=',')
    inputPath=f'.\headlinesData\{outlet}'
    outputPath=f'.\headlinesDataWithEmotionLabelsAnnotationsFromDistilRoberta\{outlet}'
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
    files=os.listdir(inputPath)
    for file in files:
        year=file.split('.')[0]
        headlinesAll=[]
        urlsAll=[]
        emotionsLabelsAll=[]
        if startYear <= int(year) <= endYear:
            print(year,end=',')
            inputFilePath=os.path.join(inputPath,file)
            df=pd.read_csv(inputFilePath)
#             for rowIndex, row in list(df.iterrows())[:25]:
            for rowIndex in range(0,len(df),BATCH_SIZE):
                try:
                    dfBatch=df[rowIndex:rowIndex+BATCH_SIZE]
                    headlinesBatch=list(dfBatch['title'].values)
                    urlsBatch=list(dfBatch['url'].values)
                    headlinesToUrlsDict={}
                    for headlineIndex, headline in enumerate(headlinesBatch):
                        headlinesToUrlsDict[headline]=urlsBatch[headlineIndex]
                        
                    headlinesBatchFiltered = [headline for headline in headlinesBatch if (type(headline)==str and len(headline.split())<=HEADLINE_MAX_LENGTH)]
                    urlsBatchFiltered=[headlinesToUrlsDict[headline] for headline in headlinesBatchFiltered]
                    emotionsLabelsBatchFiltered=get_emotion(headlinesBatchFiltered,USE_GPU=USE_GPU)
                    
                    headlinesAll += headlinesBatchFiltered 
                    urlsAll += urlsBatchFiltered 
                    emotionsLabelsAll += emotionsLabelsBatchFiltered
                    
                except Exception as e:
                    print("Something happened in sentence labeling loop", e)
                    continue
            dfOut = pd.DataFrame(list(zip(emotionsLabelsAll,headlinesAll,urlsAll)), columns =['emotion', 'headline','url']) 
            outputFilePath=os.path.join(outputPath,year+'.csv')
            dfOut.to_csv(outputFilePath,index=False)
    print()

In [7]:
outlets = list(outletsBiasRatings.keys())
startYear = 2000
endYear = 2019

for outlet in outlets:
    annotateEmotionsInOutletHeadlines(outlet,startYear,endYear)

alternet,2012,2013,2014,2015,2016,2017,2018,2019,
democracynow,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
db,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
hp,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
theintercept,2014,2015,2016,2017,2018,2019,
jacobin,2011,2012,2013,2014,2015,2016,2017,2018,2019,
motherjones,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
thenewyorker,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
thenation,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
slate,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,
vox,2014,2015,2016,2017,2018,2019,
cnn,2011,2012,2013,2014,2015,2016,2017,2018,2019,
nyt,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2